In [1]:
import torch
from PINN import PINN
from Net import Net
import numpy as np
import pandas as pd

In [2]:
data_dir = "D:/andyh/Documents/Projects/mines/methane_project/dev_jia/pinn_dev/data/ADED2022/pinn_test_data"
# load in data
df_wind_ch4 = pd.read_csv(data_dir + "/wind_ch4.csv")
df_true_emission = pd.read_csv(data_dir + "/selected_controll_release.csv")
source_points = np.load(data_dir + "/source_points.npy") # shape=(n_source, 3)
sensor_points = np.load(data_dir + "/sensor_points.npy") # shape=(n_sensor, 3)
#col_points = np.load(data_dir + "/col_points.npy")  # shape=(n_col, 3)
df_bounds = pd.read_csv(data_dir + "/bounds.csv", dtype='float32')
x_min = df_bounds['x_min'][0]
x_max = df_bounds['x_max'][0]
y_min = df_bounds['y_min'][0]
y_max = df_bounds['y_max'][0]
z_min = df_bounds['z_min'][0]
z_max = df_bounds['z_max'][0]


ws = df_wind_ch4['wind_speed.m/s'].to_numpy() # shape=(N_t,)
wd = df_wind_ch4['wind_direction'].to_numpy() # shape=(N_t,)
ch4 = np.transpose(df_wind_ch4.iloc[:, 3:].to_numpy()) # shape=(N_obs, N_t)
sensor_names = df_wind_ch4.columns[3:]

In [3]:
model = PINN([300,300,300,300])
model.set_location(source_points,[100,x_max,y_max,z_max],source_values=[0,10,0,0,0])
tfinal = 100.

In [4]:
optimizer = torch.optim.Adam(model.net.parameters(), lr=4e-2)
from torch.optim.lr_scheduler import ExponentialLR

scheduler = ExponentialLR(optimizer, gamma=0.999)  # Decay LR by 5% every epoch

In [5]:
n=2000
sn = 1000
best_loss = np.inf
for epoch in range(30000):
    if epoch % 2 == 0:
        source_points,uv_points,source_values = model.source_points(30,1)
        ic_col = torch.cat([torch.rand(sn,1)*0, torch.rand(sn,1)*x_max, torch.rand(sn,1)*y_max, torch.rand(sn,1)*z_max], dim=1)
        ic_col.requires_grad_()
        collocation_points = torch.cat([torch.rand(n,1)*tfinal, torch.rand(n,1)*x_max, torch.rand(n,1)*y_max, torch.rand(n,1)*z_max], dim=1)

    ic_val = model.forward(ic_col)
    collocation_points = torch.cat([collocation_points,ic_col])
    # collocation_points.requires_grad=True

    uv = torch.ones(len(collocation_points),2)*.5
    optimizer.zero_grad()

    loss_1 ,pde_1 = model.loss_function(collocation_points,uv)
    loss_2,pde_2 = model.loss_function(source_points, uv_points, source_values)
    loss_3 = torch.square(ic_val)

    loss = loss_1 + loss_2*10+loss_3
    loss.backward()
    if loss.item() < best_loss:
        torch.save(model,'best_mod.pth')
    optimizer.step()
    scheduler.step()

    if epoch % 100 == 0:
        print(epoch, loss.item())


RuntimeError: grad can be implicitly created only for scalar outputs

In [17]:
model = torch.load('best_mod.pth')

In [18]:
import matplotlib.pyplot as plt

# load the best model 
Z_value = 2
# net.load_state_dict(torch.load('best_model.pth'))

# Define the grid and time steps
n= 100
x_grid = np.linspace(0, x_max, n)
y_grid = np.linspace(0, y_max, n)
z_grid = np.linspace(0, 2, n)

X, Y, = np.meshgrid(x_grid, y_grid)
Z= X * 0 + Z_value

grid_points = np.vstack([X.ravel(), Y.ravel(), Z.ravel()]).T
# grid_points = np.vstack([X.ravel(), Y.ravel(), Z.ravel()]).T  # Flatten the grid
grid_points = torch.tensor(grid_points, dtype=torch.float32)

time_steps = np.linspace(0, tfinal, 20)  # 10 time steps from 0 to 1


'''
# Plot the concentration over time
for t in time_steps:
    t_tensor = torch.full((grid_points.shape[0], 1), t, dtype=torch.float32)  # Time input
    concentration = net(grid_points, t_tensor).cpu().detach().numpy().reshape(100, 100)  # Predict and reshape
    
    plt.figure()
    plt.contourf(X, Y, concentration, levels=50, cmap='viridis', vmin=0, vmax=10)  # Plot concentration as a contour plot
    plt.plot(source_loc[0,0].cpu(), source_loc[0,1].cpu(), 'ro', label='Source Location')  # Plot the source location
    plt.colorbar(label='Concentration')
    # fix colorbar from 0 to 1 
    # plt.clim(0, 10.0)  # Set colorbar limits
    plt.title(f"Gas Concentration at t = {t:.2f}")
    plt.xlabel('x')
    plt.ylabel('y')
    # fix colorbar 
    # plt.savefig(f"concentration_t_{t:.2f}.png")  # Save the plot as an image
'''

'\n# Plot the concentration over time\nfor t in time_steps:\n    t_tensor = torch.full((grid_points.shape[0], 1), t, dtype=torch.float32)  # Time input\n    concentration = net(grid_points, t_tensor).cpu().detach().numpy().reshape(100, 100)  # Predict and reshape\n    \n    plt.figure()\n    plt.contourf(X, Y, concentration, levels=50, cmap=\'viridis\', vmin=0, vmax=10)  # Plot concentration as a contour plot\n    plt.plot(source_loc[0,0].cpu(), source_loc[0,1].cpu(), \'ro\', label=\'Source Location\')  # Plot the source location\n    plt.colorbar(label=\'Concentration\')\n    # fix colorbar from 0 to 1 \n    # plt.clim(0, 10.0)  # Set colorbar limits\n    plt.title(f"Gas Concentration at t = {t:.2f}")\n    plt.xlabel(\'x\')\n    plt.ylabel(\'y\')\n    # fix colorbar \n    # plt.savefig(f"concentration_t_{t:.2f}.png")  # Save the plot as an image\n'

In [19]:
# save as a GIF
import imageio
import os

images = []
source_loc = torch.tensor([[.5,.5,.5]])
for t in time_steps:
    t_tensor = torch.full((grid_points.shape[0], 1), t, dtype=torch.float32)  # Time input
    concentration = model.forward(torch.cat([t_tensor,grid_points],dim=1),scaled=False).cpu().detach().numpy().reshape(100, 100)  # Predict and reshape
    
    plt.figure()
    plt.contourf(X, Y, concentration, levels=50, cmap='viridis', vmin=0, vmax=2)  # Plot concentration as a contour plot
    plt.plot(model.source_locs[1,0], model.source_locs[1,1], 'ro', label='Source Location')  # Plot the source location
    plt.colorbar(label='Concentration')
    plt.title(f"Gas Concentration at t = {t:.2f}")
    plt.xlabel('x')
    plt.ylabel('y')
    plt.savefig(f"concentration_t_{t:.2f}.png")  # Save the plot as an image
    plt.close()
    
    images.append(imageio.imread(f"concentration_t_{t:.2f}.png"))  # Append the image to the list
    os.remove(f"concentration_t_{t:.2f}.png")  # Remove the image file

imageio.mimsave(f'test_visual.gif', images)  # Save the images as a GIF



C:\Users\andyh\AppData\Local\Temp\ipykernel_32324\4059532269.py:21: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(f"concentration_t_{t:.2f}.png"))  # Append the image to the list


In [23]:
print(len(model.net.hidden))

5
